# Pre-Info

More Info About Certain Attributes. 

1) CRIM : Crime rate per person in each town.This attribute represents the rate of crimes committed per individual in each town. It provides information about the level of criminal activity within the area.

2) CHAS : Binary variable that determines whether a tract in each town is situated along the boundary of the Charles River. It takes a value of 1 if the tract touches the river and 0 otherwise. It indicates the proximity of the town to the river.

3) ZN : Percentage of residential land allocated for large-sized lots.

4) INDUS: Percentage of non-retail business land in each town.

5) NOX : Level of concentration of nitric oxides in the air, measured in parts per 10 million.

6) RM : Mean number of rooms found in residential dwellings within each town

7) AGE : Percentage of owner-occupied units built prior to 1940.

8) DIS : Weighted distances to five employment centers in Boston.

9) RAD : Represents an index that quantifies the ease of access from each town to radial highways

10) TAX : Property tax rate based on full property value per $10,000.

Further Information About TAX: https://www.rocketmortgage.com/learn/property-tax

11) PTRATIO : pupil-teacher ratio by town.

12) B : 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

13) LSTAT: % lower status of the population.

14) MEDV : Median value of owner-occupied homes in $1000's.*

In [ ]:
from pathlib import Path
import re

import numpy as np 
from scipy import stats
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm.auto as tqdm

In [ ]:
file_names = list(Path('/kaggle/input').rglob('*.csv'))
file = file_names[0]

In [ ]:
df = pd.read_csv(file)

# Exploratory Data Analysis (EDA) / Descriptive Method

In [ ]:
# Initially declaring the target feature, as it will be utilised further down the line.
target = 'MEDV'

In [ ]:
# There are multiple features that have a high distriubtion from the median, signaling a high right/left skew.
# This will be noted as unskew strategies will be applied later on.
df.describe()

In [ ]:
# CHAS Feature is binary, however, it is not indicated as such in the original dataset
df.dtypes

In [ ]:
# There are multiple feature that are missing data, as, fortunately, 
# all the features are numerical a median fill approach can be utilized.
df.info()

In [ ]:
df.nunique()

In [ ]:
def plot_hist(df):
    # Selecting numerical features, this is not required as all features
    # are numerical
    cols = (
        df
        .select_dtypes(include=[int, float])
        .columns
    )

    ncols = 2
    # Determining how many rows will be required based on the number of features
    # that are available; this process is agile and won't require modifications
    # if new features will be added
    nrows = np.ceil(len(cols) / ncols).astype(int)
    # The more features there are available the larger the figure will become
    vertical_figsize = 2 * nrows

    axs = plt.subplots(nrows, ncols, figsize=[10, vertical_figsize])[1].flatten()

    for col, ax in zip(cols, axs):
        df[col].plot.hist(title=col, ax=ax)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_hist(df)

As previously, predicted there are multiple features that are highly unskewed; in order to improve model performance, remove outliers and make data more interpretable.

# Processing Data

In [ ]:
# As previously metnioned, NaN values will be substituted with median
df = df.fillna(df.median(axis=0))

In [ ]:
df['CHAS'] = df['CHAS'].astype(bool)

## Feature Construction

CRIM_ZN: This interaction term captures the relationship between the per capita crime rate and the proportion of residential land zoned for lots over 25,000 sq.ft. It explores how crime rates may be influenced by the availability of large-sized residential lots.

INDUS_CHAS: This interaction term examines the relationship between the proportion of non-retail business acres per town and the presence of the Charles River. It investigates how the presence of the river may affect the proportion of non-retail business land in the area.

NOX_DIS: This interaction term explores the relationship between nitric oxides concentration in the air and the weighted distances to five major employment centers in Boston. It investigates how air pollution levels may vary based on the proximity to employment centers.

RM_AGE: This interaction term examines the relationship between the average number of rooms per dwelling and the proportion of owner-occupied units built prior to 1940. It explores how the number of rooms in a dwelling may be influenced by the age of owner-occupied units.

RAD_TAX: This interaction term investigates the relationship between the index of accessibility to radial highways and the full-value property-tax rate per $10,000. It explores how the accessibility to radial highways may impact the property tax rates in the area.

PTRATIO_B: This interaction term explores the relationship between the pupil-teacher ratio by town and a term involving the proportion of black residents. It investigates how the demographic composition of the town, specifically the proportion of black residents, may interact with the pupil-teacher ratio in shaping the educational environment.

In [ ]:
df['CRIM_ZN'] = df['CRIM'] * df['ZN']
df['INDUS_CHAS'] = df['INDUS'] * df['CHAS']
df['NOX_DIS'] = df['NOX'] * df['DIS']
df['RM_AGE'] = df['RM'] * df['AGE']
df['RAD_TAX'] = df['RAD'] * df['TAX']
df['PTRATIO_B'] = df['PTRATIO'] * df['B']

## Unskewing data

In [ ]:
skew_res = df.select_dtypes([int, float]).skew().abs().sort_values(ascending=False)
skew_cols = skew_res.loc[lambda x: (x>=1) & (x.index!=target)].index

print(skew_res)
print('-'*50)
print('Cols that are skewed:')
print(', '.join(skew_cols))

In [ ]:
def best_transformation(data) -> tuple:
    functions = [np.log1p, np.sqrt, stats.yeojohnson]
    results = []
    
    for func in functions:
        transformed_data = func(data)
        if type(transformed_data) == tuple:
            vals, _ = transformed_data
            results.append(vals)
        else:
            results.append(transformed_data)
            
    abs_skew_results = [np.abs(stats.skew(val)) for val in results]
    lowest_skew_index = abs_skew_results.index(min(abs_skew_results))
    return functions[lowest_skew_index], results[lowest_skew_index]

In [ ]:
def unskew(col):
    global best_transformation
    print('-'*100)
    col_skew = stats.skew(col)
    col_name = col.name
    print('{} skew is: {}'.format(col_name, col_skew))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10, 4])
    col.plot.hist(color='red', alpha=.4, label='pre-skew', ax=ax1)

    if np.abs(col_skew) >= 1.:
        result_skew, data = best_transformation(col)
        new_col_skew = stats.skew(data)
        print(f'Best function {result_skew} and the skew results: {new_col_skew}')
        ax2.hist(data, label='post skew processing')
        plt.legend()
        plt.show()
        
        if np.abs(new_col_skew) >= 1.:
            print(f'Transformation was not successful for {col_name}, returning original data')
            return col 
        
        return data 
    
    plt.show()
    
df[skew_cols] = df[skew_cols].apply(unskew, axis=0);

## Attribute Link Selection

Selecting highest correlated data with the target

In [ ]:
corr_ranking = (
    df
    .drop(target, axis=1)
    .corrwith(df[target])
    .abs()
    .sort_values(ascending=False)
)

In [ ]:
_, ax = plt.subplots(figsize=[10, 10])
sns.heatmap(df.corr(), annot=True, cmap="rocket", cbar=False, ax=ax);

Correlation coefficients whose magnitude are between 0.9 and 1.0 indicate variables which can be considered very highly correlated. Correlation coefficients whose magnitude are between 0.7 and 0.9 indicate variables which can be considered highly correlated. Correlation coefficients whose magnitude are between 0.5 and 0.7 indicate variables which can be considered moderately correlated. Correlation coefficients whose magnitude are between 0.3 and 0.5 indicate variables which have a low correlation. Correlation coefficients whose magnitude are less than 0.3 have little if any (linear) correlation. We can readily see that 0.9 < |r| < 1.0 corresponds with 0.81 < r2 < 1.00; 0.7 < |r| < 0.9 corresponds with 0.49 < r2 < 0.81; 0.5 < |r| < 0.7 corresponds with 0.25 < r2 < 0.49; 0.3 < |r| < 0.5 corresponds with 0.09 < r2 < 0.25; and 0.0 < |r| < 0.3 corresponds with 0.0 < r2 < 0.09.


In [ ]:
# Taking all the values above 0.3

threshold = .3
chosen_cols = corr_ranking[corr_ranking>=threshold]
print(chosen_cols)
chosen_cols = chosen_cols.index.to_list()

## Data Preparation / Machine Learning
Training test data splitting

In [ ]:
X = df[chosen_cols]
y = df[target]

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Scaling

In [ ]:
X_train.dtypes

In [ ]:
# Since CHAS did not pass correlation filtration there is no need to remove it
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols = X_train.select_dtypes([float, int]).columns.to_list()

X_train[cols] = scaler.fit_transform(X_train)
X_test[cols] = scaler.transform(X_test)

# Model Construction (Regression) / Non-Descriptive Method

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
n_features = X_train.shape[1]

In [ ]:
linear_reg_params = {
    'fit_intercept': [True, False],
}

In [ ]:
random_forest_params = {
    'n_estimators': np.sort(np.random.default_rng().choice(500, size=10, replace=False)),
    'max_features': np.sort(np.random.default_rng().choice(n_features, size=5, replace=False)),
    'max_depth': [1, 5, 10],
}

In [ ]:
xgb_params = {
    'objective': ['reg:squarederror'],
    'max_depth': [2, 5,],
    'min_child_weight': np.arange(1, 5, 2),
    'n_estimators': np.sort(np.random.default_rng().choice(500, size=3, replace=False)),
    'learning_rate': [1e-1, 1e-2,],
    'gamma': np.sort(np.random.default_rng().choice(20, size=3, replace=False)),
    'reg_lambda': [0, 1.0, 10.0],
    'scale_pos_weight': [1, 3, 5],
    'n_jobs': [-1],
}

In [ ]:
## Warning: this takes long to train, check below for best params

# models = [LinearRegression(), RandomForestRegressor(), XGBRegressor()]
# params = [linear_reg_params, random_forest_params, xgb_params]

# hgs_models = []

# for model, param in zip(models, params):
#     print(model)
#     hgs = HalvingGridSearchCV(model, param, scoring='neg_mean_squared_error', n_jobs=-1)
#     hgs.fit(X_train, y_train)
#     hgs_models.append(hgs)

In [ ]:
best_mode_params = {
    LinearRegression(): {'fit_intercept': True},
    RandomForestRegressor(): {'max_depth': 10, 'max_features': 9, 'n_estimators': 378},
    XGBRegressor(): {'gamma': 18, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 461, 'n_jobs': -1, 'objective': 'reg:squarederror', 'reg_lambda': 0, 'scale_pos_weight': 1},
}

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
b_models = []
model_results = []

for model in best_mode_params.keys():
    params = best_mode_params[model]
    model.set_params(**params)
    model.fit(X_train, y_train)    
    b_models.append(model)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    model_name = re.search(r'\w+', str(model))[0]
    results = pd.Series({'MSE': mse, 'RMSE': rmse, 'R2': r2}, name=model_name)
    model_results.append(results)

# Evaluation

In [ ]:
pd.concat(model_results, axis=1)

As visible from above, XGBoost outperformed RandomForest. 

In [ ]:
feature_imp = []
for model in b_models:
    try:
        model_name = re.search(r'\w+', str(model))[0]
        feature_imp.append(
            pd.Series(
                {
                    col: importance 
                    for col, importance in zip(cols, model.feature_importances_)
                },
                name = model_name
            )
        )
    except AttributeError:
        pass
    
pd.concat(feature_imp, axis=1).sort_values(by='XGBRegressor', ascending=False)

RM surprisingly surpassed LSTAT in terms of feature importance, despite LSTAT showing a stronger correlation with the target. Currently, it is feasible to provide a straightforward visual representation of the model's performance by graphing the actual data against the model's forecasts.

In [ ]:
xgb_model = b_models[2]
col = 'LSTAT'
y_pred = xgb_model.predict(X_test.sort_values(by=col))

(
    pd.concat([X_test[col], y_test], axis=1)
    .sort_values(by=col)
    .plot(x=col, y='MEDV', color='red', alpha=.4, label='Actual Data'),
)

plt.scatter(X_test[col].sort_values(), y_pred, label='Prediction Data')
plt.legend()
plt.show()

# Final Thoughts

The data in this notebook has undergone a comprehensive analysis and processing procedure, leading to the development of reliable models. The evaluation results demonstrate the effectiveness of these models in generating accurate predictions. However, there is still potential for further improvement. To address the problem more effectively, it would be advantageous to utilize a tool that compares different models before conducting hyperparameter tuning. This approach would enable the selection of the optimal model and enhance the overall performance of the predictive models

- A Descriptive Method has been implemented
- A Non-Descriptive
- Machine Learning has been applied
- Appopriate Security measures have been taken. 
- Interface to view data and go throughout the parsing set has been shown. 